In [1]:
import torch
print(torch.__version__)

2.5.0+cu121


In [2]:
!pip install torch==2.2.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [1]:
import torch
print(torch.__version__)

2.2.1+cu121


In [2]:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.2.1+cu121.html

Looking in links: https://data.pyg.org/whl/torch-2.2.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 943.4/943.4 kB 25.4 MB/s eta 0:00:00


In [3]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.4 MB/s eta 0:00:00


In [35]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from torch_geometric.nn import to_hetero
from torch_geometric.datasets import IMDB
from torch_geometric.loader import NeighborLoader
from torch.nn.functional import dropout
from torch_geometric.nn import GCNConv, Linear, SAGEConv
from torch.optim import Adam

In [21]:
class GCN(torch.nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.conv1 = SAGEConv((-1, -1), 64)
        self.conv2 = SAGEConv((-1, -1), 128)

        self.linear1 = Linear(128, 64)
        self.linear2 = Linear(64, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).tanh()
        x = dropout(x, p=0.3)

        x = self.conv2(x, edge_index).tanh()
        x = dropout(x, p=0.3)

        x = self.linear1(x)
        x = self.linear2(x)

        return x

In [22]:
base_model = GCN(num_classes=3)

In [23]:
base_model

GCN(
  (conv1): SAGEConv((-1, -1), 64, aggr=mean)
  (conv2): SAGEConv((-1, -1), 128, aggr=mean)
  (linear1): Linear(128, 64, bias=True)
  (linear2): Linear(64, 3, bias=True)
)

In [9]:
data = IMDB('IMDB')

Extracting IMDB/raw/IMDB_processed.zip
Processing...
Done!


In [12]:
dataset = data[0]
dataset

HeteroData(
  movie={
    x=[4278, 3066],
    y=[4278],
    train_mask=[4278],
    val_mask=[4278],
    test_mask=[4278],
  },
  director={ x=[2081, 3066] },
  actor={ x=[5257, 3066] },
  (movie, to, director)={ edge_index=[2, 4278] },
  (movie, to, actor)={ edge_index=[2, 12828] },
  (director, to, movie)={ edge_index=[2, 4278] },
  (actor, to, movie)={ edge_index=[2, 12828] }
)

In [16]:
set(dataset['movie'].y.detach().cpu().numpy())

{0, 1, 2}

In [19]:
dataset.metadata()

(['movie', 'director', 'actor'],
 [('movie', 'to', 'director'),
  ('movie', 'to', 'actor'),
  ('director', 'to', 'movie'),
  ('actor', 'to', 'movie')])

In [24]:
model = to_hetero(base_model, dataset.metadata(), aggr='sum')

/usr/local/lib/python3.10/dist-packages/torch_geometric/nn/fx.py:132: UserWarning: Found function 'dropout' with keyword argument 'training'. During FX tracing, this will likely be baked in as a constant value. Consider replacing this function by a module to properly encapsulate its training flag.
  warnings.warn(f"Found function '{node.name}' with keyword "
/usr/local/lib/python3.10/dist-packages/torch_geometric/nn/fx.py:132: UserWarning: Found function 'dropout_1' with keyword argument 'training'. During FX tracing, this will likely be baked in as a constant value. Consider replacing this function by a module to properly encapsulate its training flag.
  warnings.warn(f"Found function '{node.name}' with keyword "


In [25]:
model

GraphModule(
  (conv1): ModuleDict(
    (movie__to__director): SAGEConv((-1, -1), 64, aggr=mean)
    (movie__to__actor): SAGEConv((-1, -1), 64, aggr=mean)
    (director__to__movie): SAGEConv((-1, -1), 64, aggr=mean)
    (actor__to__movie): SAGEConv((-1, -1), 64, aggr=mean)
  )
  (conv2): ModuleDict(
    (movie__to__director): SAGEConv((-1, -1), 128, aggr=mean)
    (movie__to__actor): SAGEConv((-1, -1), 128, aggr=mean)
    (director__to__movie): SAGEConv((-1, -1), 128, aggr=mean)
    (actor__to__movie): SAGEConv((-1, -1), 128, aggr=mean)
  )
  (linear1): ModuleDict(
    (movie): Linear(128, 64, bias=True)
    (director): Linear(128, 64, bias=True)
    (actor): Linear(128, 64, bias=True)
  )
  (linear2): ModuleDict(
    (movie): Linear(64, 3, bias=True)
    (director): Linear(64, 3, bias=True)
    (actor): Linear(64, 3, bias=True)
  )
)

In [26]:
train_input_nodes = ('movie', dataset['movie'].train_mask)
train_loader = NeighborLoader(dataset, num_neighbors=[3, 3],
                              shuffle=True, input_nodes=train_input_nodes,
                              batch_size=128)

In [29]:
train_input_nodes

('movie', tensor([False, False, False,  ..., False, False, False]))

In [32]:
next(iter(train_loader))

HeteroData(
  movie={
    x=[774, 3066],
    y=[774],
    train_mask=[774],
    val_mask=[774],
    test_mask=[774],
    n_id=[774],
    num_sampled_nodes=[3],
    input_id=[128],
    batch_size=128,
  },
  director={
    x=[126, 3066],
    n_id=[126],
    num_sampled_nodes=[3],
  },
  actor={
    x=[355, 3066],
    n_id=[355],
    num_sampled_nodes=[3],
  },
  (movie, to, director)={
    edge_index=[2, 280],
    e_id=[280],
    num_sampled_edges=[2],
  },
  (movie, to, actor)={
    edge_index=[2, 802],
    e_id=[802],
    num_sampled_edges=[2],
  },
  (director, to, movie)={
    edge_index=[2, 128],
    e_id=[128],
    num_sampled_edges=[2],
  },
  (actor, to, movie)={
    edge_index=[2, 384],
    e_id=[384],
    num_sampled_edges=[2],
  }
)

In [27]:
val_input_nodes = ('movie', dataset['movie'].val_mask)
val_loader = NeighborLoader(dataset, num_neighbors=[3, 3],
                            shuffle=False, input_nodes=val_input_nodes,
                            batch_size=128)

In [28]:
test_input_nodes = ('movie', dataset['movie'].test_mask)
test_loader = NeighborLoader(dataset, num_neighbors=[3, 3],
                             shuffle=False, input_nodes=test_input_nodes,
                             batch_size=128)

In [33]:
from torch.nn.functional import one_hot


def train_classification(model,
                         train_loader, val_loader,
                         optimizer, criterion,
                         epochs=5):
    for epoch in range(epochs):
        for i, batch in enumerate(train_loader):
            model.train()
            optimizer.zero_grad()

            x_dict = batch.x_dict
            edge_index_dict = batch.edge_index_dict
            train_mask = batch['movie'].train_mask
            data_y = one_hot(batch['movie'].y, 3).float()

            out = model(x_dict, edge_index_dict)
            loss = criterion(out['movie'][train_mask], data_y[train_mask])
            train_loss = loss.item()

            loss.backward()
            optimizer.step()

            print(f'Epoch: {epoch:03d}, Step: {i:03d}, Loss: {train_loss:.4f}')

        for i, batch in enumerate(val_loader):
            model.eval()

            x_dict = batch.x_dict
            edge_index_dict = batch.edge_index_dict
            val_mask = batch['movie'].val_mask
            data_y = one_hot(batch['movie'].y, 3).float()

            out = model(x_dict, edge_index_dict)
            loss = criterion(out['movie'][val_mask], data_y[val_mask])
            val_loss = loss.item()

            print(f'Epoch: {epoch:03d}, Step: {i:03d}, Val Loss: {val_loss:.4f}')

In [36]:
optimizer = Adam(model.parameters(), lr=0.001)

In [37]:
criterion = CrossEntropyLoss()

In [38]:
train_classification(model=model,
                     train_loader=train_loader, val_loader=val_loader,
                     optimizer=optimizer, criterion=criterion,
                     epochs=5)

Epoch: 000, Step: 000, Loss: 1.0977
Epoch: 000, Step: 001, Loss: 1.0898
Epoch: 000, Step: 002, Loss: 1.0777
Epoch: 000, Step: 003, Loss: 1.0641
Epoch: 000, Step: 000, Val Loss: 1.0963
Epoch: 000, Step: 001, Val Loss: 1.0475
Epoch: 000, Step: 002, Val Loss: 1.0399
Epoch: 000, Step: 003, Val Loss: 0.9975
Epoch: 001, Step: 000, Loss: 1.0285
Epoch: 001, Step: 001, Loss: 1.0169
Epoch: 001, Step: 002, Loss: 1.0142
Epoch: 001, Step: 003, Loss: 1.0588
Epoch: 001, Step: 000, Val Loss: 1.0727
Epoch: 001, Step: 001, Val Loss: 1.0011
Epoch: 001, Step: 002, Val Loss: 1.0000
Epoch: 001, Step: 003, Val Loss: 0.9199
Epoch: 002, Step: 000, Loss: 0.9299
Epoch: 002, Step: 001, Loss: 0.8959
Epoch: 002, Step: 002, Loss: 0.8675
Epoch: 002, Step: 003, Loss: 0.8424
Epoch: 002, Step: 000, Val Loss: 1.0308
Epoch: 002, Step: 001, Val Loss: 0.9539
Epoch: 002, Step: 002, Val Loss: 0.9561
Epoch: 002, Step: 003, Val Loss: 0.8534
Epoch: 003, Step: 000, Loss: 0.7359
Epoch: 003, Step: 001, Loss: 0.6986
Epoch: 003, Step